원문: Moler, Cleve B. [Numerical computing with MATLAB](https://www.mathworks.com/moler/chapters.html). Society for Industrial and Applied Mathematics, 2008.

# 1.5 암호학

이 절에서는 암호학 예제로 문자열을 다루는 법을 설명한다. 해당 암호화 기법, *힐 암호 Hill cipher* 는 *유한체 finite field* 에서의 계산을 포함한다.
거의 모든 현대적인 컴퓨터들은 ASCII 문자 집합으로 기본 문장을 저장한다. ASCII 란 *미국 정보 교환 표준 코드 Americal Standard Code for Information Exchange* 라는 뜻이다. 해당 문자 집합은 한 바이트 안 8개의 비트 중 7개를 사용하여 128 문자를 코드로 표시한다. 첫 32글자는 표시되지 않는 제어문자로 탭, 백스페이스, 줄 끝 표시 등이다. 128번째 글자는 또하나의 표시되지 않는 문자인데 여러분 키보드의 삭제 Delete 키에 대응된다. 이러한 제어 문자열 사이에 95자의 표시되는 문자가 자리하는데, 공백, 10진수, 소문자 26자, 대문자 26자와 문장부호 32자를 포함한다.
다음과 같이 표시되는 문자를 ASCII 순서대로 표시할 수 있다.
우선 3 행 16열로 정수 행렬을 준비하여 표시한다.

In [ ]:
import numpy as np
import pprint

x = np.arange(32, 127 + 1, dtype=np.uint8).reshape((3, 32))
pprint.pprint(x, width=160)

그 다음 아래와 같이 문자열로 표시할 수 있다.

In [ ]:
# A. Lella, "Convert Numpy array of ASCII codes to string", Stackoverflow.com, 2014. [Online]. Available: https://stackoverflow.com/questions/24838409/convert-numpy-array-of-ascii-codes-to-string. [Accessed: 05- Aug- 2017].
c_list = [''.join([chr(x_int) for x_int in x_row]) for x_row in x]
pprint.pprint(c_list)

마지막 문자 127 은 원래는 표시되지 않지만 여기서는 \x7f 로 16진수 형태로 표시되었다.
``c_list`` 의 첫 문자는 공백이다. 다음 명령문의 결과를 확인하라.

In [ ]:
chr(32)

``c_list``에서 표시되는 마지막 문자는 *물결무늬 tilde* 이다. 역시 다음 결과를 확인하라.

In [ ]:
chr(126)

숫자를 표시하는 문자들은 ``c_list``의 첫줄에 있다.

In [ ]:
c_list[0]

아래와 같이 표시할 수도 있다

In [ ]:
c_list[0][16:(25+1)]

또 다른 방법으로는 다음과 같이 가능하다.

In [ ]:
d_list = [chr(i) for i in range(48,(57+1))]

위 `list` 는 숫자로 바꿀 수도 있다.

In [ ]:
d_list_float = [float(d) - float('0') for d in d_list]
d_list_float

다음 같이 하는 방법도 가능하다.

In [ ]:
# J. Kington, "How to convert an array of strings to an array of floats in numpy?", Stackoverflow.com, 2017. [Online]. Available: https://stackoverflow.com/questions/3877209/how-to-convert-an-array-of-strings-to-an-array-of-floats-in-numpy. [Accessed: 05- Aug- 2017].
d_array = np.array(d_list).astype(int) - int('0')
d_array

`c_list`의 두째 세째 행을 비교해 보면 소문자의 ASCII 코드는 해당 대문자의 ASCII 코드에 32를 더한 것이라는 것을 알 수 있다

In [ ]:
print(c_list[1])
print(c_list[2])

이러한 인코딩 encoding을 잘 이해하면 벡터와 행렬 연산으로 문장을 다룰 수 있게 된다.
ASCII 표준은 때때로 확장되어 한 바이트의 8비트 모두를 이용하기도 하지만, 사용하는 컴퓨터나 OS의 종류, 선택한 글자형, 살고 있는 나라에 따라서 다르다. 다음을 시도해 보라.

In [ ]:
import numpy as np
import pprint

x = np.array([chr(i) for i in range(128, 255+1)]).reshape((-1, 32))
pprint.pprint(x, width=160)

우리의 암호화 기법은 *[합동 산술](https://ko.wikipedia.org/wiki/합동_산술) [modular arithmetic](http://mathworld.wolfram.com/ModularArithmetic.html)* 을 이용한다. 관련된 모든 값은 정수이고 어떤 산술 연산 결과도 어떤 솟수 $p$의 나머지로 취해진다. 다음과 같이 계산할 수 있다. 

In [ ]:
import math
import numpy as np

x = np.random.randint(0, 1024)
y = 17
print('%d %% %d == %d' % (x, y, (x % y)))
print('np.mod(%d, %d) == %d' % (x, y, np.mod(x, y)))

우리는 문자 만이 아니라 ASCII 문자 집합 전체를 사용하여 문장을 암호화할 것이다. 그러한 문자는 95가지가 있다. 그 다음 큰 솟수는 $p=97$ 이므로, 우리는 $p$개의 문자를 정수 $0, 1, \cdots (p-1)$로 표시하고 $p$에 대한 나머지를 취할 것이다.


In [ ]:
import numpy as np

p=97

문자들은 한번에 2개씩 인코드 될 것이다. 각 문자 쌍은 길이 2 짜리 벡터 $x$로 표현 될 것이다. 예를 들어 `'TV'` 를 나타내는 문장을 생각해 보라. 이 쌍을 위한 해당 ASCII 값은 84와 86 이다. 시작 값을 0으로 만들기 위해 32를 빼면 열 벡터는 다음과 같다

In [ ]:
# tkf, "Character <-> Int conversion in numpy", gist.github.com, 2012. [Online]. Available: https://gist.github.com/tkf/2276773. [Accessed: 05- Aug- 2017].

import numpy as np

x_char = np.array(('T', 'V'), dtype='|S1').reshape((2, 1))
print(repr(x_char))
x_ascii = x_char.view(np.int8)
print(repr(x_ascii))
x = x_ascii + (-32)
x